In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModelWithProjection

model = CLIPVisionModelWithProjection.from_pretrained(
    pretrained_model_name_or_path='openai/clip-vit-large-patch14',
    ignore_mismatched_sizes=True
)
model.to(device='cuda:0')

In [ ]:
import src.data.make_dataset as md
from torch.utils.data import DataLoader
from src import utils
import src.models.clip.make_lightning as clip_ml

config = utils.get_config()

wandb_config = utils.load_config('clip.yml')
model = clip_ml.get_model(wandb_config)
base_dataset = md.get_base_dataset(config)
train_indices, val_indices = md.get_train_val_indices(wandb_config, base_dataset)

dataset = md.RefConDataset(
    config=config,
    wandb_config=wandb_config,
    dataset=base_dataset,
    indices=train_indices,
    train=True
)

dataloader = DataLoader(
    dataset=dataset,
    batch_size=wandb_config['batch_size'],
    num_workers=config['archi']['num_workers'],
    drop_last=True,
    shuffle=True,
)

In [ ]:
from src.submissions import search
from PIL import Image
from src import utils
import torch

config = utils.get_config()
wandb_run = utils.get_run('2khs9u4f')
model = search.RefConInferenceModel(config, wandb_run, 0)
image = Image.open('/home/external-rosia/RosIA/reflection-connection/data/raw/train/Boring/abwao.png')
image = image.convert("RGB")
outputs = model(images=image)

del model
torch.cuda.empty_cache()

In [ ]:
from src.submissions import search
from PIL import Image
from src import utils
import torch

config = utils.get_config()
wandb_run = utils.get_run('2khs9u4f')

query_set = search.ImageSet(config, wandb_run, query=True, cuda_idx=0)
corpus_set = search.ImageSet(config, wandb_run, query=False, cuda_idx=0)
query_set.build_embeddings()
corpus_set.build_embeddings()
sbf = search.SearchBruteForce(corpus_set)
sbf.query(query_set)